<a href="https://colab.research.google.com/github/jbaremoney/amm_problem/blob/main/amm_12552.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AMM 12552 recursive solution
problem def'n:\
Let $A=\{0,1,2\}$. For $n\ge 0$, define
\
$T_n=\{\,x\in A^n : \text{'20' does not occur in } x\,\}$.\
ie $T_2=\{00,01,02,10,11,12,21,22\}$

Find the sum of 1's over all x in $T_n$

In [1]:
import itertools
from functools import lru_cache


# the brute force stuff
def count_valid(strings):
    return sum("20" not in s for s in strings)

def total_ones_over_valid(strings):
    return sum(s.count('1') for s in strings if "20" not in s)

ALPHABET = "012"

def generate_Tn(n: int, as_set: bool = False):
    """
    Return all length-n strings over {0,1,2}.
    If as_set=True, return a set; otherwise return a list (lexicographic order).
    """
    if n < 0:
        raise ValueError("n must be nonnegative")
    it = (''.join(t) for t in itertools.product(ALPHABET, repeat=n))
    return set(it) if as_set else list(it)


Recursive def'n proposed: \
 $f(n)$ = number of valid strings \
                          $g(n)$ = total number of ones over the whole set
                          
                          f(n+1) = 3f(n) - f(n-1)
                          g(n+1) = 3g(n) + f(n) - g(n-1)

h(f,n+1) = f(n) - f(n-1) --> h(g, n+1) = g(n) -

In [2]:
def num_valid_strings(n: int) -> int:
    # S(n) base cases
    if n == 0: return 1       # empty string
    if n == 1: return 3       # "0","1","2"
    return 3*num_valid_strings(n-1) - num_valid_strings(n-2)

@lru_cache(None)
def num_ones(n: int) -> int:
    # O(n) base cases
    if n == 0: return 0
    if n == 1: return 1       # only "1" contributes
    return 3*num_ones(n-1) + num_valid_strings(n-1) - num_ones(n-2)

Testing - closed form proposed as follows for num of valid strings

$f(n)=\frac{5+3\sqrt{5}}{10}\left(\frac{3+\sqrt{5}}{2}\right)^{n}
\;+\;\frac{5-3\sqrt{5}}{10}\left(\frac{3-\sqrt{5}}{2}\right)^{n}$

In [7]:
from math import sqrt

def closed_form(n):
  """closed form of recurrence relation of f, number of valid strings for given n"""
  res = ((5+3*sqrt(5))/10)*(((3+ sqrt(5))/2)**n) + ((5-3*sqrt(5))/10) * (((3-sqrt(5))/2)**n)
  return res

for n in range(0, 6):  # brute force is O(3^n)
    Tn = generate_Tn(n)
    brute_valid = count_valid(Tn)
    brute_ones  = total_ones_over_valid(Tn)
    rec_valid   = num_valid_strings(n)
    rec_ones    = num_ones(n)
    closed_ones = closed_form(n)
    print(f"n={n}: valid  brute={brute_valid:>4}  rec={rec_valid:>4} closed={closed_ones:>4} | ones  brute={brute_ones:>4}  rec={rec_ones:>4}")


# Example single n
n = 3
Tn = generate_Tn(n)
print("\nExample:")
print("count_valid(Tn)       =", count_valid(Tn))        # expected 21 for n=3
print("num_valid_strings(3)  =", num_valid_strings(3))   # 21
print("total_ones_over_valid =", total_ones_over_valid(Tn))
print("num_ones(3)           =", num_ones(3))            # 25

n=0: valid  brute=   1  rec=   1 | ones  brute=   0  rec=   0 closed= 1.0
n=1: valid  brute=   3  rec=   3 | ones  brute=   1  rec=   1 closed=3.0000000000000004
n=2: valid  brute=   8  rec=   8 | ones  brute=   6  rec=   6 closed=8.000000000000002
n=3: valid  brute=  21  rec=  21 | ones  brute=  25  rec=  25 closed=21.000000000000004
n=4: valid  brute=  55  rec=  55 | ones  brute=  90  rec=  90 closed=55.000000000000014
n=5: valid  brute= 144  rec= 144 | ones  brute= 300  rec= 300 closed=144.00000000000003

Example:
count_valid(Tn)       = 21
num_valid_strings(3)  = 21
total_ones_over_valid = 25
num_ones(3)           = 25


Closed form for number of valid strings works, now just need close form for number of 1's